In [1]:
import numpy as np
import time
import itertools
import hashlib

file_name = 'wordlist.txt'

anagram = 'poultry outwits ants'
anagram = anagram.replace(' ', '')

solution_hashes = ['e4820b45d2277f3844eac66c903e84be',
                   '23170acc097c24edb98fc5488ab033fe',
                   '665e5bcb0c20062fe8abaaf4628bb154']

with open(file_name, 'r') as file:
    words_original = file.read().split('\n')
    if '' in words_original:
        words_original.remove('')

print('Number of words in list:', len(words_original))

Number of words in list: 99175


In [2]:
# helper functions
def check_hash(solutions, target_hashes):
    hashes = [hashlib.md5(solution.encode()).hexdigest() for solution in solutions]

    for target_hash in target_hashes:
        try:
            print(f'"{solutions[hashes.index(target_hash)]}" matches hash {target_hash}')
        except:
            pass
#             print(target_hash, 'not found in solution')

def words_to_phrases(word_lst):
    phrases = []
    for words in word_lst:
        n_words = len(words)
        for permutation in itertools.permutations(words, n_words):
            phrase = ' '.join(permutation)
            phrases.append(phrase)
    return phrases

print('\nwords_to_phrases', ['this', 'is', 'sparta'])
print(words_to_phrases(word_lst=[['this', 'is', 'sparta']]))

def get_character_count(word):
    char_cnt = {}
    for char in set(word):
        char_cnt[char] = word.count(char)
    
    return char_cnt

print('\nget_character_count:', anagram)
print(get_character_count(word=anagram))

def is_anagram(anagram, phrase):
    ana_cnt = get_character_count(anagram)
    phrase_cnt = get_character_count(phrase)
    
    if ana_cnt == phrase_cnt:
        return True
    else:
        return False
    
print('\nis_anagram:', 'anagram', 'margana')
print(is_anagram(anagram='anagram', phrase='margana'))


words_to_phrases ['this', 'is', 'sparta']
['this is sparta', 'this sparta is', 'is this sparta', 'is sparta this', 'sparta this is', 'sparta is this']

get_character_count: poultryoutwitsants
{'n': 1, 'u': 2, 'y': 1, 'l': 1, 'o': 2, 'p': 1, 'i': 1, 'a': 1, 'w': 1, 's': 2, 'r': 1, 't': 4}

is_anagram: anagram margana
True


# Word list filtering
1. Only keep unique words
2. The word is required to only contain characters that are in the anagram phrase
3. The word cannot have more instances of a given character than is within the anagram phrase

In [3]:
# Bullet 1 - filter out non-unique words
def remove_non_unique(words):
    return set(words)

print(f'Words in current list:', len(words_original))
words = remove_non_unique(words_original)
print('Words remaining:', len(words))

Words in current list: 99175
Words remaining: 96317


In [4]:
# Bullet 3 - ignore words with too many instances of given character       
def filter_execive_char_words(words, anagram):
    # character count in anagram
    anagram_char_cnt = get_character_count(anagram)

    # keep only words with character count compatible with anagram
    words_cnt = []
    for word in words:
        keep_word = True
        for char in set(word):
            char_cnt = word.count(char)
            if char_cnt > anagram_char_cnt.get(char, 0):
                keep_word = False
                break

        if keep_word:
            words_cnt.append(word)
            
    return words_cnt
        
print(f'Words in current list:', len(words))
words = filter_execive_char_words(words, anagram)
print('Words remaining:', len(words))

Words in current list: 96317
Words remaining: 1659


## Word scoring
Each word is given a score based on their contained characters. Each character has a unique value, that is then summed up for the word. 

In [5]:
# provide each word a numerical score
def get_value_map(chars):
    value_map = {}
    for idx, char in enumerate(chars):
        value_map[char] = 10**idx
        
    return value_map
        
def get_word_value(word, value_map):
    value = 0
    
    for char in word:
        value += value_map[char]
            
    return value


def assign_word_values(words, value_map):
    word_values = {}

    for word in words:
        word_value = get_word_value(word, value_map=value_map)
        word_values[word] = word_value
    
    sorted_value_list = sorted(word_values.items(), key=lambda t: t[::-1])

    return dict(sorted_value_list), sorted_value_list

value_map = get_value_map(set(anagram))
word_value_dct, sorted_value_list = assign_word_values(words, value_map)
anagram_value = get_word_value(word=anagram, value_map=value_map)

print('Anagram phrase:', anagram)
print('Anagram value:', anagram_value)

# checking uniqueness of word values
print('\nChecking uniqueness of word value assignments...')
print(f'Number of unique words:', len(np.unique(np.array([sorted(word) for word in words], dtype=object))))
print(f'Number of unique word values:', len(np.unique([val[1] for val in sorted_value_list])))


Anagram phrase: poultryoutwitsants
Anagram value: 412111121121

Checking uniqueness of word value assignments...
Number of unique words: 1179
Number of unique word values: 1179


# Solution for 2 words
Instead of looping through all words twice to find if their values sum up to the target value (O(n^2)), we can use a sorted list and only go through the list once (O(n)). We look at pairs of numbers by combining low and high values each time and moving the current index according to the target value.

In [6]:
# 2 element sums
def get_pair_sum(word_list, sum_target):
    solutions = []

    idx_low = 0
    idx_high = len(word_list) - 1
    
    while idx_low < idx_high:
        val_low = word_list[idx_low][1]
        val_high = word_list[idx_high][1]

        phrase_value = val_low + val_high
        
        if phrase_value == sum_target:
            word_low = word_list[idx_low][0]
            word_high = word_list[idx_high][0]
            
            solutions.append([word_low, word_high])
    
            # this logic keeps the loop running for mutiple words with same characters: ex: nip, pin
            next_value_low = word_list[idx_low + 1][1]
            next_value_high = word_list[idx_high - 1][1]
            
            if next_value_low == val_low:
                idx_low += 1
            elif next_value_high == val_high:
                idx_high -= 1
            else:
                idx_low += 1

        elif phrase_value < sum_target:
            idx_low += 1
        else:
            idx_high -= 1

    return solutions

# function test: expected ('a', 'f'), ('b', 'e') and ('c', 'e')
get_pair_sum(word_list=[('a', 3), ('b', 4), ('c', 4), ('d',8), ('e', 11), ('f', 12), ('g', 15)], sum_target=15)

[['a', 'f'], ['b', 'e'], ['c', 'e']]

# Solution 
The algorithm scans through all the words in the available word list, for each required word in the solution phrase. However, when reaching the last two words, the above algorithm is used, reducing the time complexity.
The solution allows for the same word to appear multiple times.

The solution can look for phrases with any number of word, in theory. However, as it has a time complexity of O(n^(N-1)), with n being the length of the word list, and N being the number of words in the phrase, it is exponential and quickly becomes infeasible. The solution also stores all potential phrases, which will limit memory at some point.

It is implemented using recursion, with the base case being a 2 word phrase, using the O(n) algorithm from above.

In [7]:
def get_anagram_words(word_lst, target_value, n_words_anagram, words_current=[]):
    solutions_total = []
    
    if n_words_anagram == 1:
        for word, value in word_lst:
            if value == target_value:
                solutions_total.append([word])
            elif value > target_value:
                break
                
    elif n_words_anagram == 2:
        word_pairs = get_pair_sum(word_list=word_lst, sum_target=target_value)
        
        if len(word_pairs) > 0:
            for pair in word_pairs:
                solutions_total.append(words_current + pair)
    else:
        n_words_anagram -= 1
        idx_lst = 0 # if this is 0, then the same word can appear multiple times, if 1 then it cannot
            
        for word, value in word_lst:
            solutions = get_anagram_words(word_lst[idx_lst:],
                                          target_value=target_value - value,
                                          n_words_anagram=n_words_anagram,
                                          words_current=words_current + [word]
                                         )
            
            if solutions is not None:
                for solution in solutions:
                    solutions_total.append(solution)
                
            idx_lst += 1
    
    return solutions_total


for n in range(1, 5):
    print(f'\nLooking for solutions with {n} words')
    t0 = time.time()
    solutions = get_anagram_words(word_lst=sorted_value_list,
                      target_value=anagram_value,
                      n_words_anagram=n,
                      words_current=[])


    anagram_phrases = words_to_phrases(solutions)
    print(f'Valid anagram phrases with {n} words:', len(anagram_phrases))
    check_hash(anagram_phrases, target_hashes=solution_hashes)
    print(f'Done in {time.time() - t0:.2f} seconds')


Looking for solutions with 1 words
Valid anagram phrases with 1 words: 0
Done in 0.00 seconds

Looking for solutions with 2 words
Valid anagram phrases with 2 words: 0
Done in 0.00 seconds

Looking for solutions with 3 words
Valid anagram phrases with 3 words: 4554
"printout stout yawls" matches hash e4820b45d2277f3844eac66c903e84be
"ty outlaws printouts" matches hash 23170acc097c24edb98fc5488ab033fe
Done in 0.18 seconds

Looking for solutions with 4 words
Valid anagram phrases with 4 words: 6980712
"wu lisp not statutory" matches hash 665e5bcb0c20062fe8abaaf4628bb154
Done in 100.72 seconds


# Optimized algorithm
A small optimization of the solution above. The complexity is still exponential, but the following optimizations are implemented:
1. The initial wordlist now only contains uniquely valued words, meaning "pin", "nip" will only appear once. The words are expanded into all valid form after the algorithm has run.
2. The word list used for each new word iteration is filtered such that only words that are still valid are scanned. Validity is determined as words that only contain the remaining available characters of the anagram phrase at its current state. The available characters of the input phrase is being tracked at each word iteration.


In [8]:
def filter_execive_char_words(words, anagram_char_remainder):
    # keep only words with character count compatible with characters currently left in the anagram phrase
    words_cnt = []
    for word, value in words:
        keep_word = True
        for char in set(word):
            char_cnt = word.count(char)
            if char_cnt > anagram_char_remainder.get(char, 0):
                keep_word = False
                break

        if keep_word:
            words_cnt.append((word, value))
            
    return words_cnt

def anagram_character_counter(word, char_remainder):
    # keeps track of characters left in the anagram phrase
    new_char_remainder = char_remainder.copy()
    for char in word:
        new_char_remainder[char] -= 1
    
    return new_char_remainder

# pre algo runtime word filtering
def filter_rendundant_words(words):
    # Only keep a single instance of a words characters
    words_lookup_dct = {}
    for word in words:
        w_sorted = ''.join(sorted(word))

        if w_sorted in words_lookup_dct:
            w_lst = words_lookup_dct[w_sorted]
            w_lst.append(word)
            words_lookup_dct[w_sorted] = w_lst
        else:
            words_lookup_dct[w_sorted] = [word]
    
    
    words_unique = [word for word in words_lookup_dct]
    
    return words_unique, words_lookup_dct

# Post algo runtime word to phrase expansion. Gets all combinations of different variation of words with the same characters
# and gets the phrases available with those words
def expand_words_to_phrase(words_lst, word_dct):
    phrases = []
    for words in words_lst:
        word_lst = [word_dct[word] for word in words]
        word_combinations = list(itertools.product(*word_lst))
        word_phrases = words_to_phrases(word_combinations)
        
        for phrase in word_phrases:
            phrases.append(phrase)
            
    return phrases


def get_anagram_words_optimized(word_lst, target_value, char_remainder, n_words_anagram, words_current=[]):
    solutions_total = []
    
    if n_words_anagram == 1:
        for word, value in word_lst:
            if value == target_value:
                solutions_total.append([word])
            elif value > target_value:
                break
                
    elif n_words_anagram == 2:
        word_pairs = get_pair_sum(word_list=word_lst, sum_target=target_value)
        if len(word_pairs) > 0:
            for pair in word_pairs:
                solutions_total.append(words_current + pair)
    else:
        n_words_anagram -= 1
        idx_lst = 0 # if this is 0, then the same word can appear multiple times, if 1 then it cannot
 
        for word, value in word_lst:
            new_char_remainder = anagram_character_counter(word=word, char_remainder=char_remainder)
            word_lst_new = filter_execive_char_words(words=word_lst[idx_lst:], anagram_char_remainder=new_char_remainder)

            solutions = get_anagram_words_optimized(word_lst_new,
                                                    target_value=target_value - value,
                                                    char_remainder = new_char_remainder,
                                                    n_words_anagram=n_words_anagram,
                                                    words_current=words_current + [word]
                                                   )
            
            if solutions is not None:
                for solution in solutions:
                    solutions_total.append(solution)
                
            idx_lst += 1
    
    return solutions_total

print('Words current:', len(words))
words_unique, words_lookup_dct = filter_rendundant_words(words)
word_value_dct, sorted_value_list = assign_word_values(words_unique, value_map)
print('Words remaining:', len(words_unique))


for n in range(1, 5):
    print(f'\nLooking for solutions with {n} words')
    t0 = time.time()
    solutions = get_anagram_words_optimized(word_lst=sorted_value_list,
                                            target_value=anagram_value,
                                            char_remainder=get_character_count(word=anagram),    
                                            n_words_anagram=n,
                                            words_current=[]
                                           )

    print('Expanding words into phrases')
    anagram_phrases = expand_words_to_phrase(words_lst=solutions, word_dct=words_lookup_dct)
    print(f'Valid anagram phrases with {n} words:', len(anagram_phrases))
    check_hash(anagram_phrases, target_hashes=solution_hashes)
    print(f'Done in {time.time() - t0:.2f} seconds')

Words current: 1659
Words remaining: 1179

Looking for solutions with 1 words
Expanding words into phrases
Valid anagram phrases with 1 words: 0
Done in 0.12 seconds

Looking for solutions with 2 words
Expanding words into phrases
Valid anagram phrases with 2 words: 0
Done in 0.00 seconds

Looking for solutions with 3 words
Expanding words into phrases
Valid anagram phrases with 3 words: 4560
"printout stout yawls" matches hash e4820b45d2277f3844eac66c903e84be
"ty outlaws printouts" matches hash 23170acc097c24edb98fc5488ab033fe
Done in 0.57 seconds

Looking for solutions with 4 words
Expanding words into phrases
Valid anagram phrases with 4 words: 7427232
"wu lisp not statutory" matches hash 665e5bcb0c20062fe8abaaf4628bb154
Done in 26.76 seconds
